In [1]:
import os
import argparse
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset,DataLoader

import torchvision.utils as utils
import torchvision.transforms as transforms
import torchvision.models as models


import copy
import glob
import torch.utils.data as udata
import h5py
import cv2
import random
import pywt
import matplotlib.pyplot as plt
from PIL import Image
import math
from skimage.measure.simple_metrics import compare_psnr
import scipy.io as sio
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #Anaconda内的一个.dll和pytorch的一个libiomp5md.dll冲突了，仅需要允许备份存在即可
plt.rcParams['font.sans-serif'] = [u'SimHei'] #正常输出中文字符 黑体
plt.rcParams['axes.unicode_minus'] = False #正常输出正负号

In [2]:
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor #可以用gpu的话，数据类型是torch.cuda.FloatTensor 否则使用torch.FloatTensor
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# desired size of the output image
imsize = 128  # use small size if no gpu 图片大小，这是学长的看法，我觉得都可以用128

#有imsize,imsize 但应该不会改变
loader = transforms.Compose([ 
    transforms.Resize([imsize, imsize]),  # scale imported image 变换图片为对应大小
    transforms.ToTensor()])  # transform it into a torch tensor  转化为张量
def image_loader(image_name): #数据加载 读进图片进行加载
    image = Image.open(image_name)  #打开图片
    image = Variable(loader(image)) #图片从张量变成变量，Variable是网络参数优化的必须数据类型，可以对梯度进行计算保存
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0) #在第一维度之后再加一个维度 ？不知道为啥
    return image


In [3]:
LOAD_PATH = "./l2-1" #加载数据库数据路径
# STYLE_PATH = "1_zero.jpg"

In [4]:
img_lst = os.listdir(LOAD_PATH) #
img_lst.sort() #排序路径下所有文件或文件夹

In [5]:
#访问路径下所有图片，并且把图片和图片对应名称加入这个列表
res_lst = []
name_lst = []
# a = image_loader(STYLE_PATH).type(dtype).to(device)
for img_name in img_lst:
    if os.path.splitext(img_name)[-1] != '.png':
        continue
    img_path = os.path.join(LOAD_PATH, img_name)#加入图片
    img = image_loader(img_path).type(dtype).to(device)  #图片加载

    
    res_lst.append(img.to('cpu').numpy()) #图片资源列表 
    name_lst.append(img_name) #图片名称列表


In [6]:
res = np.asarray(res_lst) # 转化为数组
res = res.squeeze() #加入一个维度
name_lst = np.asarray(name_lst) #转化为数组
mat = {'img':res, 'img_name':name_lst}

In [7]:
sio.savemat('l2-1', mat)